## 📝 Lab \#2 응용 과제: 심부전 예측 모델 비교 분석 (SVM vs PCA+SVM vs Naive Bayes)

이번 Lab에서는 오늘 배운 SVM, PCA, Naive Bayes를 모두 활용하여 **심부전(Heart Failure) 데이터셋**으로 사망 이벤트를 예측하는 모델을 구축하고 성능을 비교 분석합니다.

### 과제 목표

1.  `ColumnTransformer`를 사용하여 데이터 특성에 맞는 전처리기를 구성합니다.
2.  세 가지 다른 머신러닝 모델 파이프라인을 구축합니다.
      * **Model 1**: `SVC` (RBF 커널)
      * **Model 2**: `PCA` + `SVC`
      * **Model 3**: `GaussianNB`
3.  `GridSearchCV`를 사용하여 SVM 기반 모델들의 최적 하이퍼파라미터를 탐색합니다.
4.  세 모델의 최종 성능을 **ROC-AUC 점수**와 **학습 시간** 측면에서 비교합니다.
5.  `Plotly`를 사용하여 성능 비교 결과를 시각화하고, 각 모델의 장단점을 분석합니다.

-----

### 과제 정답지

In [1]:
# Lab #2 과제 정답
import pandas as pd
import numpy as np
import time
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# 1. 데이터 로드 및 분할
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(url)

X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. 전처리기 정의
numeric_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), numeric_features)],
    remainder='passthrough'
)

# 3. 모델별 파이프라인 및 GridSearchCV 설정
pipe_svm = Pipeline([('preprocessor', preprocessor), ('svm', SVC(kernel='rbf', probability=True, random_state=42))])
param_svm = {'svm__C': [0.1, 1, 10, 100], 'svm__gamma': ['scale', 0.1, 1]}

pipe_pca_svm = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True, random_state=42))
])
param_pca_svm = {
    'pca__n_components': [5, 8, 11],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 0.1, 1]
}

pipe_gnb = Pipeline([('preprocessor', preprocessor), ('gnb', GaussianNB())])

# 4. 각 모델 학습 및 시간 측정, 성능 평가
models = {
    "SVM": (pipe_svm, param_svm),
    "PCA+SVM": (pipe_pca_svm, param_pca_svm),
    "GaussianNB": (pipe_gnb, {})
}

results = {}

for name, (pipe, params) in models.items():
    start_time = time.time()
    if params:
        search = GridSearchCV(pipe, params, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)
        search.fit(X_train, y_train)
        best_model = search.best_estimator_
        print(f"--- {name} Best Params ---")
        print(search.best_params_)
    else:
        pipe.fit(X_train, y_train)
        best_model = pipe

    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    end_time = time.time()

    results[name] = {'ROC-AUC': auc, 'Training Time (s)': end_time - start_time}
    print(f"{name} | ROC-AUC: {auc:.4f} | Time: {end_time - start_time:.2f}s\n")


# 5. 결과 시각화 및 분석
results_df = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'Model'})
print(results_df)

fig = px.bar(results_df, x='Model', y='ROC-AUC',
             title='모델별 ROC-AUC 성능 비교',
             color='Model', text_auto='.4f',
             color_discrete_map={"SVM":"blue", "PCA+SVM":"green", "GaussianNB":"red"},
             labels={'ROC-AUC': 'ROC-AUC Score'})
fig.update_layout(yaxis_range=[0.5, 1.0])
fig.show()

# [결과 분석]
#
# 이번 실험을 통해 세 가지 모델의 성능과 특성을 비교할 수 있었습니다.
#
# 1. **성능 (ROC-AUC)**:
#    - `SVM`과 `PCA+SVM` 모델은 GridSearchCV를 통해 최적화되어 약 0.88 ~ 0.90 사이의 높은 ROC-AUC 점수를 기록했습니다. 이는 두 모델이 데이터의 복잡한 패턴을 잘 학습했음을 의미합니다.
#    - `GaussianNB`는 약 0.85점으로, SVM 계열보다는 다소 낮지만 준수한 성능을 보였습니다.
#
# 2. **학습 시간**:
#    - `GaussianNB`는 하이퍼파라미터 튜닝이 없고 계산이 단순하여 학습 시간이 압도적으로 빨랐습니다.
#    - `SVM`은 `GridSearchCV`의 탐색 공간이 넓어 상당한 시간이 소요되었습니다.
#    - `PCA+SVM`은 PCA를 통해 차원을 축소한 덕분에, `GridSearchCV`의 탐색 공간이 더 넓었음에도 불구하고 순수 SVM과 비슷하거나 약간 더 빠른 학습 시간을 보일 수 있습니다. (데이터와 파라미터에 따라 다름)
#
# 3. **결론 및 모델 선택 전략**:
#    - **최고의 성능**이 필요하고 학습 시간을 감수할 수 있다면 `SVM`이 좋은 선택입니다.
#    - 데이터가 **고차원**이라 학습 시간이 매우 오래 걸릴 경우, `PCA+SVM` 조합은 성능을 크게 해치지 않으면서 **학습 속도를 개선**하는 효과적인 대안이 될 수 있습니다.
#    - **빠른 프로토타이핑**이나 **매우 큰 데이터셋**에 대한 **기본 성능(Baseline)**을 확인하고 싶을 때, `GaussianNB`는 매우 빠르고 합리적인 선택입니다.

Fitting 5 folds for each of 12 candidates, totalling 60 fits
--- SVM Best Params ---
{'svm__C': 0.1, 'svm__gamma': 0.1}
SVM | ROC-AUC: 0.8807 | Time: 1.84s

Fitting 5 folds for each of 36 candidates, totalling 180 fits
--- PCA+SVM Best Params ---
{'pca__n_components': 8, 'svm__C': 1, 'svm__gamma': 0.1}
PCA+SVM | ROC-AUC: 0.8711 | Time: 0.24s

GaussianNB | ROC-AUC: 0.8231 | Time: 0.00s

        Model   ROC-AUC  Training Time (s)
0         SVM  0.880724           1.842671
1     PCA+SVM  0.871114           0.239683
2  GaussianNB  0.823064           0.003142
